In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error

In [2]:
#Reading dataset
df = pd.read_csv("C:/Users/Rishi/Amazon_Recommendation_System/dataset.csv")

In [3]:
#Analyzing
print(df.columns)
print(df.shape)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
(568454, 10)


In [4]:
count = df.groupby("ProductId", as_index=False).count()
mean = df.groupby("ProductId", as_index=False).mean()

In [5]:
dfMerged = pd.merge(df, count, how='right', on=['ProductId'])
dfMerged

,Id_x,ProductId,UserId_x,ProfileName_x,HelpfulnessNumerator_x,HelpfulnessDenominator_x,Score_x,Time_x,Summary_x,Text_x,Id_y,UserId_y,ProfileName_y,HelpfulnessNumerator_y,HelpfulnessDenominator_y,Score_y,Time_y,Summary_y,Text_y
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,1,1,1,1,1,1,1,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,1,1,1,1,1,1,1,1,1
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,1,1,1,1,1,1,1,1,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,1,1,1,1,1,1,1,1,1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...,6,6,6,6,6,6,6,6,6
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...,1,1,1,1,1,1,1,1,1
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",2,2,2,2,2,2,2,2,2
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...,2,2,2,2,2,2,2,2,2


In [6]:
dfMerged["totalReviewers"] = dfMerged["UserId_y"]
dfMerged["overallScore"] = dfMerged["Score_x"]
dfMerged["summaryReview"] = dfMerged["Summary_x"]

In [7]:
dfNew = dfMerged[['ProductId','summaryReview','overallScore',"totalReviewers"]]

In [8]:
dfMerged = dfMerged.sort_values(by='totalReviewers', ascending=False)
dfCount = dfMerged[dfMerged.totalReviewers >= 100]
dfCount

,Id_x,ProductId,UserId_x,ProfileName_x,HelpfulnessNumerator_x,HelpfulnessDenominator_x,Score_x,Time_x,Summary_x,Text_x,...,ProfileName_y,HelpfulnessNumerator_y,HelpfulnessDenominator_y,Score_y,Time_y,Summary_y,Text_y,totalReviewers,overallScore,summaryReview
563881,563882,B007JFMH8M,A9JS5GQQ6GIQT,Syne,0,0,5,1343433600,yummy,I love the Quaker soft baked cookies. The rea...,...,913,913,913,913,913,913,913,913,5,yummy
563615,563616,B007JFMH8M,A3H19949DTO802,Jessi,0,0,5,1341878400,Healthy and yummy too,We love these! They are so soft and tasty. The...,...,913,913,913,913,913,913,913,913,5,Healthy and yummy too
563629,563630,B007JFMH8M,A1ACM1CBGORBN1,Sherry Berry,0,0,4,1344124800,Soft and chewy but a bit too sweet!,I am kind of shocked that these cookies were t...,...,913,913,913,913,913,913,913,913,4,Soft and chewy but a bit too sweet!
563628,563629,B007JFMH8M,A2H2I5FY1PUHP1,Boilermate,0,0,4,1344211200,Pretty Good Cookie,"First of all, I like that the cookies are indi...",...,913,913,913,913,913,913,913,913,4,Pretty Good Cookie
563627,563628,B007JFMH8M,A10PEXB6XAQ5XF,Michael Gallagher,0,0,3,1344211200,Good Taste and Flavor But a Little on the Dry ...,This is a pretty good cookie in terms of taste...,...,913,913,913,913,913,913,913,913,3,Good Taste and Flavor But a Little on the Dry ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528752,528753,B000EVMNMI,A7G9M0IE7LABX,Kevin,0,0,4,1307059200,Great Value!,Great Value compared to buying in the store!<b...,...,100,100,100,100,100,100,100,100,4,Great Value!
11802,11803,B004P4TL8E,A1B2HCS03AC5DT,Mary P.,202,219,4,1299715200,"This is not necessarily a ""space saver""","So, I bought the Coffee Pod Drawer thinking it...",...,100,100,100,100,100,100,100,100,4,"This is not necessarily a ""space saver"""
528755,528756,B000EVMNMI,A3094EPI56GKZ6,"Kristen E. Bogren ""Kristen Bogren""",0,0,5,1289433600,Super Yummy Cola Taste!,I love Haribo Gummy candy and the cola taste i...,...,100,100,100,100,100,100,100,100,5,Super Yummy Cola Taste!
11800,11801,B004P4TL8E,A3VMRU1PX1SX22,Daniel Correia,5,11,4,1304467200,Great tray for storing various K-cup varieties,"Before I begin my review, let me say this... I...",...,100,100,100,100,100,100,100,100,4,Great tray for storing various K-cup varieties


In [11]:
dfProductReview = df.groupby("ProductId", as_index=False).mean()
ProductReviewSummary = dfCount.groupby("ProductId")["summaryReview"].apply(list)
ProductReviewSummary = pd.DataFrame(ProductReviewSummary)
ProductReviewSummary.to_csv("C:/Users/Rishi/Amazon_Recommendation_System/ProductReview.csv")

In [1]:
dfProductReview

NameError: name 'dfProductReview' is not defined

In [2]:
df3 = pd.read_csv("C:/Users/Rishi/Amazon_Recommendation_System/ProductSummary.csv")
df3 = pd.merge(df3, dfProductReview, on="ProductId", how='inner')

NameError: name 'pd' is not defined

In [3]:
df3 = df3[['ProductId','summaryReview','Score']]

NameError: name 'df3' is not defined

In [4]:
regEx = re.compile('[^a-z]+')
def cleanReviews(Text):
    Text = Text.lower()
    Text = regEx.sub(' ', Text).strip()
    return Text

NameError: name 're' is not defined

In [5]:
df3["summaryClean"] = df3["summaryReview"].apply(cleanReviews)
df3 = df3.drop_duplicates(['Score'], keep='last')
df3 = df3.reset_index()

NameError: name 'df3' is not defined

In [6]:
reviews = df3["summaryClean"] 
countVector = CountVectorizer(max_features = 300, stop_words='english') 
transformedReviews = countVector.fit_transform(reviews) 

NameError: name 'df3' is not defined

In [7]:
dfReviews = DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)

NameError: name 'DataFrame' is not defined

In [8]:
dfReviews.to_csv("C:/Users/Rishi/Amazon_Recommendation_System/dfReviews.csv")

NameError: name 'dfReviews' is not defined

In [9]:
X = np.array(dfReviews)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]
#len of train and test
lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

NameError: name 'np' is not defined

In [10]:
print(lentrain)
print(lentest)
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(dfReviews_train)
distances, indices = neighbor.kneighbors(dfReviews_train)

NameError: name 'lentrain' is not defined

In [11]:
for i in range(lentest):
    a = neighbor.kneighbors([dfReviews_test[i]])
    related_product_list = a[1]

    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Based on product reviews, for ", df3["ProductId"][lentrain + i] ," average rating is ",df3["Score"][lentrain + i])
    print ("The first similar product is ", df3["ProductId"][first_related_product] ," average rating is ",df3["Score"][first_related_product])
    print ("The second similar product is ", df3["ProductId"][second_related_product] ," average rating is ",df3["Score"][second_related_product])
    print ("-----------------------------------------------------------")
df5_train_target = df3["overallScore"][:lentrain]
df5_test_target = df3["overallScore"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

NameError: name 'lentest' is not defined

In [12]:
n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

NameError: name 'neighbors' is not defined

In [13]:
print(classification_report(df5_test_target, knnpreds_test)) 
print (accuracy_score(df5_test_target, knnpreds_test))

NameError: name 'classification_report' is not defined

In [14]:
print(mean_squared_error(df5_test_target, knnpreds_test)) 

NameError: name 'mean_squared_error' is not defined

In [ ]:
knn.score(df5_test_target, knnpreds_test)
acc = accuracy_score(df5_test_target, knnpreds_test)